In [1]:
import re
import pandas as pd
from datetime import datetime
wd = '/Volumes/PEDS/RI Biostatistics Core/Shared/Shared Projects/Vigers/CF/Christine Chan/Prepost Triakfta/'

In [2]:
# CHCO data
chco_dates = pd.read_csv(wd + 'Data_Cleaned/chco_trikafta_dates.csv')
# Fix selected dates
chco_dates.loc[chco_dates.MRN == 1053930,'Start'] = "12/1/2019"
chco_dates.loc[chco_dates.MRN == 1695512,'Start'] = "12/12/2019"
chco_dates.loc[chco_dates.MRN == 815274,'Start'] = "12/13/2019"
chco_dates.loc[chco_dates.MRN == 1078408,'Start'] = "3/9/2020"
chco_dates.loc[chco_dates.MRN == 899717,'Start'] = "2/14/2020"
# CFRD only
chco_dates = chco_dates.loc[chco_dates['CFRD yes=1'] == 1]
# Remove those without Trikafta date
chco_dates.dropna(subset = ['Start'],inplace = True)
# Add glycemic data
chco_a1c = pd.read_csv(wd + 'Data_Cleaned/chco_a1c_ogtt.csv')
chco = pd.merge(chco_dates,chco_a1c,on = ['MRN'],how = 'left')
# Add PFTs
chco_pfts = pd.read_csv(wd + 'Data_Cleaned/chco_bmi_pft.csv')
idx = chco_pfts['MRN'].isin(chco_dates['MRN']) 
chco_pfts = chco_pfts[idx]
chco = pd.merge(chco,chco_pfts,on = ['MRN','Date'],how = 'outer')
# Add CGM
chco_cgm = pd.read_csv(wd + 'Data_Cleaned/chco_cgm.csv')
chco_cgm['CFF ID'] = [int(re.sub('_.*','',s)) for s in chco_cgm['subject_id']]
chco_cgm['Date'] = [re.sub(' .*','',s) for s in chco_cgm['date_cgm_placement']]
chco = pd.merge(chco,chco_cgm,on = ['CFF ID','Date'],how = 'outer')
# Sort
chco.sort_values(by = ['MRN','Date'],inplace = True)
# Fill missing 
fill = ['DOB','CFF ID','Start','Sex','Race','Hispanic/Latinx','Genotypes1','Genotypes2','Pancreatic Status','CFRD Diagnosis Date']
chco[fill] = chco.groupby('MRN')[fill].ffill()
chco[fill] = chco.groupby('MRN')[fill].bfill()
# Write
chco.to_csv(wd + 'Data_Cleaned/chco_final.csv',index = False)


In [3]:
# Montana data
adult_dem = pd.read_csv(wd + 'Data_Cleaned/montana_demo_adults.csv')
peds_dem = pd.read_csv(wd + 'Data_Cleaned/montana_demo_kids.csv')
montana = pd.concat([adult_dem,peds_dem])
# Add CGM
montana_cgm = pd.read_csv(wd + 'Data_Cleaned/montana_cgm.csv')
montana_cgm['CFF ID'] = [int(re.sub('_.*','',s)) for s in montana_cgm['subject_id']]
montana_cgm['Date'] = [re.sub(' .*','',s) for s in montana_cgm['date_cgm_placement']]
montana = pd.merge(montana,montana_cgm,on = ['CFF ID'],how = 'outer')
# Add A1c and OGTTs
montana_a1c = pd.read_csv(wd + 'Data_Cleaned/montana_a1c_ogtt.csv')
montana_a1c.dropna(subset = ['CFRD Status'],inplace = True)
montana = pd.merge(montana,montana_a1c,on = ['CFF ID','Date'],how = 'outer')
# Add PFTs
montana_pfts_adult = pd.read_csv(wd + 'Data_Cleaned/montana_bmi_pft_adults.csv')
montana_pfts_peds = pd.read_csv(wd + 'Data_Cleaned/montana_bmi_pft_kids.csv')
montana_pfts = pd.concat([montana_pfts_adult,montana_pfts_peds])
montana = pd.merge(montana,montana_pfts,on = ['CFF ID','Date'],how = 'outer')
# Sort
montana.sort_values(by = ['CFF ID','Date'],inplace = True)
# Fill missing 
fill = ['DOB','Start','Gender','Race','Mixed Race components','Is Patient of Hispanic Origin']
montana[fill] = montana.groupby('CFF ID')[fill].ffill()
montana[fill] = montana.groupby('CFF ID')[fill].bfill()
# Write
montana.to_csv(wd + 'Data_Cleaned/montana_final.csv',index = False)

In [19]:
# Washington data
uw_dates = pd.read_csv(wd + 'Data_Cleaned/uw_trikafta_dates.csv')
uw_dates.drop_duplicates(subset='CFF ID',inplace = True)
# Glycemic data
uw_a1c = pd.read_csv(wd + 'Data_Cleaned/uw_a1c_ogtt.csv')
uw = pd.merge(uw_dates,uw_a1c,on = ['CFF ID'],how = 'outer')
# BMI
uw_bmi = pd.read_csv(wd + 'Data_Cleaned/uw_bmi.csv')
uw = pd.merge(uw,uw_bmi,on = ['CFF ID','Date'],how = 'outer')
# PFTs
uw_pft = pd.read_csv(wd + 'Data_Cleaned/uw_pft.csv')
uw = pd.merge(uw,uw_pft,on = ['CFF ID','Date','Patients height'],how = 'outer')
# Write
uw.to_csv(wd + 'Data_Cleaned/uw_final.csv',index = False)

In [20]:
uw

,CFF ID,Start,HbA1c,OGTT Fasting,OGTT Two Hour,Date,BMI Percentile,Encounter Age,Patients height,BMI Value,Measure of FEV1,Patients height.1,Measure of FVC,Predicted Value for FVC,Predicted Value for FEV1,Predicted Value for FEF25-75
0,1440650,12/2/19,11.7,NaN,NaN,5/10/19,14.79,19.6,160.0,NaN,1.67,160.0,2.24,3.66,3.23,3.70
1,1440650,12/2/19,12.4,NaN,NaN,10/4/19,NaN,20.0,160.0,19.57,2.17,160.0,2.69,3.66,3.23,3.69
2,1440650,12/2/19,13.5,NaN,NaN,7/31/20,NaN,20.9,160.0,20.78,2.43,160.0,3.05,3.66,3.22,3.67
3,1440650,12/2/19,12.8,NaN,NaN,3/1/19,9.09,19.4,160.0,NaN,1.96,160.0,2.50,3.66,3.23,3.71
4,1440650,12/2/19,14.0,NaN,NaN,6/5/20,NaN,20.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470,1649554,NaN,NaN,NaN,NaN,10/26/20,NaN,0.3,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2471,1649554,NaN,NaN,NaN,NaN,8/17/20,NaN,0.1,53.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2472,1649596,NaN,NaN,NaN,NaN,10/28/20,NaN,0.1,53.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2473,1649596,NaN,NaN,NaN,NaN,11/9/20,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
help(uw_dates.drop_duplicates)

Help on method drop_duplicates in module pandas.core.frame:

drop_duplicates(subset: 'Hashable | Sequence[Hashable] | None' = None, keep: "Literal['first'] | Literal['last'] | Literal[False]" = 'first', inplace: 'bool' = False, ignore_index: 'bool' = False) -> 'DataFrame | None' method of pandas.core.frame.DataFrame instance
    Return DataFrame with duplicate rows removed.
    
    Considering certain columns is optional. Indexes, including time indexes
    are ignored.
    
    Parameters
    ----------
    subset : column label or sequence of labels, optional
        Only consider certain columns for identifying duplicates, by
        default use all of the columns.
    keep : {'first', 'last', False}, default 'first'
        Determines which duplicates (if any) to keep.
        - ``first`` : Drop duplicates except for the first occurrence.
        - ``last`` : Drop duplicates except for the last occurrence.
        - False : Drop all duplicates.
    inplace : bool, default False
  